In [ ]:
pip install emoji


In [ ]:
import pandas as pd
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
import string

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from collections import Counter


In [ ]:
# Specify the path to your CSV file
csv_file_path = '/content/drive/MyDrive/NLP DATADET/twcs.csv'

# Read CSV file into DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
2811769,2987947,sprintcare,False,Wed Nov 22 08:43:51 +0000 2017,"@823869 Hey, we'd be happy to look into this f...",NaN,2987948.0
2811770,2987948,823869,True,Wed Nov 22 08:35:16 +0000 2017,@115714 wtf!? I’ve been having really shitty s...,2987947,NaN
2811771,2812240,121673,True,Thu Nov 23 04:13:07 +0000 2017,@143549 @sprintcare You have to go to https://...,NaN,2812239.0
2811772,2987949,AldiUK,False,Wed Nov 22 08:31:24 +0000 2017,"@823870 Sounds delicious, Sarah! 😋 https://t.c...",NaN,2987950.0


In [ ]:
df.info()

In [ ]:
df['text']

0          @115712 I understand. I would like to assist y...
1              @sprintcare and how do you propose we do that
2          @sprintcare I have sent several private messag...
3          @115712 Please send us a Private Message so th...
4                                         @sprintcare I did.
                                 ...                        
2811769    @823869 Hey, we'd be happy to look into this f...
2811770    @115714 wtf!? I’ve been having really shitty s...
2811771    @143549 @sprintcare You have to go to https://...
2811772    @823870 Sounds delicious, Sarah! 😋 https://t.c...
2811773    @AldiUK  warm sloe gin mince pies with ice cre...
Name: text, Length: 2811774, dtype: object

In [ ]:
df= df.sort_values(by='created_at', ascending=True)


#Lower Case Text

In [ ]:
df_lower_case = df.copy()
df_lower_case['text_lower_case']=df_lower_case['text'].str.lower()
df_lower_case

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower_case
2496505,2666295,751178,True,Fri Apr 01 17:37:48 +0000 2016,@AskSeagate Same issue Seagate. Is there a sol...,2666294,2666296.0,@askseagate same issue seagate. is there a sol...
2496504,2666294,AskSeagate,False,Fri Apr 01 18:22:40 +0000 2016,@751178 Pls give us more clarification on your...,2666293,2666295.0,@751178 pls give us more clarification on your...
2496503,2666293,751178,True,Fri Apr 01 18:25:52 +0000 2016,@AskSeagate Bootstrap and media server daemon ...,2666291,2666294.0,@askseagate bootstrap and media server daemon ...
2496501,2666291,AskSeagate,False,Fri Apr 01 19:53:33 +0000 2016,@751178 Pls use the link bellow to download th...,2666292,2666293.0,@751178 pls use the link bellow to download th...
352823,402549,211231,True,Fri Apr 03 19:57:41 +0000 2015,@AsurionCares How do I get a receipt of my mon...,402547,NaN,@asurioncares how do i get a receipt of my mon...
...,...,...,...,...,...,...,...,...
2533273,2703608,759372,True,Wed Sep 28 13:34:20 +0000 2016,@sainsburys sainsburys local - central station,2703607,2703609.0,@sainsburys sainsburys local - central station
445058,505744,236012,True,Wed Sep 28 16:03:15 +0000 2016,@NikeSupport Buenos días. La App no registró m...,505742,NaN,@nikesupport buenos días. la app no registró m...
2533272,2703607,sainsburys,False,Wed Sep 28 18:03:12 +0000 2016,"@759372 Hi Tom, this one: https://t.co/VRGF9EG...",2703606,2703608.0,"@759372 hi tom, this one: https://t.co/vrgf9eg..."
2533271,2703606,759372,True,Wed Sep 28 18:04:04 +0000 2016,@sainsburys yeah,2703604,2703607.0,@sainsburys yeah


#Analysis

Number of unique companies in the dataset

In [ ]:

# Count occurrences of each author_id
author_counts = df[df['inbound'] ==False ]['author_id'].value_counts()

# Sort author_counts by maximum count
author_counts_sorted = author_counts.sort_values(ascending=False)
print("Percentage of outbound conversations")
print(((df[df['inbound'] ==False].shape[0])/df.shape[0])*100)

print("\n\nAuthor Name\t\tCount")
print("------------------------")
for author_id, count in author_counts_sorted.items():
    print(f"{author_id}\t\t{count}")




Percentage of outbound conversations
45.30701969646209


Author Name		Count
------------------------
AmazonHelp		169840
AppleSupport		106860
Uber_Support		56270
SpotifyCares		43265
Delta		42253
Tesco		38573
AmericanAir		36764
TMobileHelp		34317
comcastcares		33031
British_Airways		29361
SouthwestAir		28977
VirginTrains		27817
Ask_Spectrum		25860
XboxSupport		24557
sprintcare		22381
hulu_support		21872
sainsburys		19466
GWRHelp		19364
AskPlayStation		19098
ChipotleTweets		18749
VerizonSupport		17966
UPSHelp		17817
ATVIAssist		17650
O2		16212
Safaricom_Care		16077
idea_cares		15724
AskTarget		13218
AirAsiaSupport		12829
BofA_Help		12683
SW_Help		12231
ArgosHelpers		12174
AskLyft		11809
marksandspencer		11504
AskAmex		11358
MicrosoftHelps		11304
AskPayPal		11298
Morrisons		10126
AdobeCare		9917
airtel_care		9881
AskeBay		9686
AirbnbHelp		8875
ChaseSupport		8811
McDonalds		8478
AldiUK		8114
JetBlue		8020
CoxHelp		7861
Ask_WellsFargo		7578
AzureSupport		7565
AlaskaAir		7478
HPSupport		7200


Extracting tweets into dialogues where begining of conversation is identified by null to_response_id and end of conversation is identified by null response_id



In [ ]:
beginning_conversations_df = df[df['in_response_to_tweet_id'].isnull()]
non_null_response_df = df[~df['in_response_to_tweet_id'].isnull()]
print(beginning_conversations_df.shape)

(794335, 7)


In [ ]:
dialogues = []
counter=0
for index, row in beginning_conversations_df.iterrows():
    tweet_id = row['tweet_id']
    response_id = row['response_tweet_id']
    conversations = [{"author": row['author_id'], "text": row['text'],
                      "response_id": row['response_tweet_id'],
                      "to_response_id": row['in_response_to_tweet_id'],
                      "created_at" : row['created_at']}]

    # Use set for faster membership testing
    seen_responses = set()
    split_response_id = response_id.split(',')
    print(counter)
    counter+=1
    while split_response_id:
        response = split_response_id.pop(0)
        if response in seen_responses:
            continue

        df_response = non_null_response_df[non_null_response_df['tweet_id'] == int(response)]
        if df_response.empty:
            continue

        conv = {"author": df_response['author_id'].iloc[0],
                "text": df_response['text'].iloc[0],
                "response_id": df_response['response_tweet_id'].iloc[0],
                "to_response_id": df_response['in_response_to_tweet_id'].iloc[0],
                "created_at" : df_response['created_at'].iloc[0]}

        conversations.append(conv)
        response_id = df_response['response_tweet_id'].iloc[0]
        if not pd.isnull(response_id):
            split_response_id.extend(response_id.split(','))

        seen_responses.add(response)

    dialogues.append(conversations)


Streaming output truncated to the last 5000 lines.
789336
789337
789338
789339
789340
789341
789342
789343
789344
789345
789346
789347
789348
789349
789350
789351
789352
789353
789354
789355
789356
789357
789358
789359
789360
789361
789362
789363
789364
789365
789366
789367
789368
789369
789370
789371
789372
789373
789374
789375
789376
789377
789378
789379
789380
789381
789382
789383
789384
789385
789386
789387
789388
789389
789390
789391
789392
789393
789394
789395
789396
789397
789398
789399
789400
789401
789402
789403
789404
789405
789406
789407
789408
789409
789410
789411
789412
789413
789414
789415
789416
789417
789418
789419
789420
789421
789422
789423
789424
789425
789426
789427
789428
789429
789430
789431
789432
789433
789434
789435
789436
789437
789438
789439
789440
789441
789442
789443
789444
789445
789446
789447
789448
789449
789450
789451
789452
789453
789454
789455
789456
789457
789458
789459
789460
789461
789462
789463
789464
789465
789466
789467
789468
789469
789470
7894

In [ ]:
for dialogue in dialogues:
    # Sort the dialogue by created_at
    dialogue.sort(key=lambda x: x["created_at"])


In [ ]:
dialogues

[[{'author': '211231',
   'text': "@AsurionCares How do I get a receipt of my monthly charges? I can't seem to find any way to do this and I need it for an expense report.",
   'response_id': '402547',
   'to_response_id': nan,
   'created_at': 'Fri Apr 03 19:57:41 +0000 2015'},
  {'author': 'AsurionCares',
   'text': '@211231 Typically, you can contact your carrier for a receipt of your monthly premiums. ^LN',
   'response_id': '402548',
   'to_response_id': 402549.0,
   'created_at': 'Fri Apr 03 20:02:54 +0000 2015'},
  {'author': '211231',
   'text': "@AsurionCares Yeah, for some reason it isn't listed on my Verizon bill and its charged separately to my Credit Card as ESH Tech Protect.",
   'response_id': '402550',
   'to_response_id': 402547.0,
   'created_at': 'Fri Apr 03 20:03:58 +0000 2015'},
  {'author': '211232',
   'text': '@211231 @AsurionCares I have same issue. How do you get Asurion to answer?',
   'response_id': '402551',
   'to_response_id': 402548.0,
   'created_at': '

Get maximum length of thread


In [ ]:
max_length = 0
dialogue_with_max_length = None

# Iterate over each array in dialogues
for dialogue in dialogues:
    # Calculate the length of the current array
    current_length = len(dialogue)

    # Update max_length if the current length is greater
    if current_length > max_length:
        max_length = current_length
        dialogue_with_max_length = dialogue

# Print the maximum length
print("Maximum length of thread in dialogue:", max_length)


Maximum length of thread in dialogue: 1390


Get maximum length of text in tweets from companies


In [ ]:
import re

# Function to check if a string contains only numbers
def contains_only_numbers(s):
    return re.match("^[0-9]+$", s) is not None

conversation_with_number_authors = None

# Iterate over each conversation in dialogues
for conversation in dialogues:
    # Check if all authors' names contain only numbers
    if all(contains_only_numbers(item["author"]) for item in conversation):
        conversation_with_number_authors = conversation
        break  # Stop iterating if found

# Check if there is a conversation with number authors
if conversation_with_number_authors is not None:
    print("Found a conversation where all authors' names contain only numbers:")
    for item in conversation_with_number_authors:
        print("Author:", item["author"])
        print("Text:", item["text"])
        print("Response ID:", item["response_id"])
        print("Created At:", item["created_at"])
        print()  # Add an empty line for better readability between items
else:
    print("No conversation found where all authors' names contain only numbers.")


No conversation found where all authors' names contain only numbers.


In [ ]:
# Filter the DataFrame to include only rows where inbound is False
filtered_df = df[df['inbound'] == False]

# Find the row with the maximum length of the "text" column
max_length_row = filtered_df.loc[filtered_df['text'].str.len().idxmax()]

# Extract text and its length
max_length_text = max_length_row['text']
max_length_text_length = len(max_length_text)

# Print the row with the maximum length of "text"
print("Row with maximum text length:")
print(max_length_row)
print()

# Print the text and its length
print("Text with maximum length:")
print("Text:", max_length_text)
print("Length:", max_length_text_length)


Row with maximum text length:
tweet_id                                                             2087231
author_id                                                    marksandspencer
inbound                                                                False
created_at                                    Mon Nov 13 16:55:19 +0000 2017
text                       @617137 @617138 @117246 @117242 @Tesco @sainsb...
response_tweet_id                                            2087233,2087234
in_response_to_tweet_id                                            2087230.0
Name: 1932740, dtype: object

Text with maximum length:
Text: @617137 @617138 @117246 @117242 @Tesco @sainsburys @117248 @AldiUK @117260 @117249 @Morrisons @117250 @117251 @117241 We'll 100% get behind the #SupermarketRoyalRumble - we'll come away greater winners than John Cena's Royal Rumble return in 2008 🙌 Hey @117246 - #YouCantSeeMe because we're so high up the leaderboard 😉 https://t.co/1QmeG4h5Jq
Length: 343


Get maximum length of text in tweets where they are talking to only one person (only one mention)

In [ ]:
import re

# Function to check if the text meets the criteria
def check_text(text):
    # Use a regular expression to check if the text matches the pattern
    return re.match(r'^[^@]*@[^@\s]+$', text) is not None

# Filter the DataFrame to include only rows where inbound is False
filtered_df = df[df['inbound'] == False]

# Filter the DataFrame further to include only rows where the text meets the criteria
filtered_df = filtered_df[filtered_df['text'].apply(check_text)]

max_length_index = filtered_df['text'].str.len().idxmax()

# Get the row with maximum text length
max_length_row = filtered_df.loc[max_length_index]

# Extract text and its length
max_length_text = max_length_row['text']
max_length_text_length = len(max_length_text)

# Print the row with the maximum length of "text"
print("Row with maximum text length:")
print(max_length_row)
print()

# Print the text and its length
print("Text with maximum length:")
print("Text:", max_length_text)
print("Length:", max_length_text_length)


Row with maximum text length:
tweet_id                                                              191443
author_id                                                     TwitterSupport
inbound                                                                False
created_at                                    Wed Jul 05 17:17:37 +0000 2017
text                       We're excited to announce that we've got a shi...
response_tweet_id          191442,191444,191445,191446,191447,191448,1914...
in_response_to_tweet_id                                                  NaN
Name: 159494, dtype: object

Text with maximum length:
Text: We're excited to announce that we've got a shiny, new username! Still your official source for help on Twitter, now from @TwitterSupport.
Length: 137


In [ ]:

# Function to check if the text meets the criteria
def check_text(text):
    # Use a regular expression to check if the text matches the pattern
    return re.match(r'^[^@]*@[^@\s]+$', text) is not None

# Filter the DataFrame to include only rows where inbound is False
filtered_df = df[df['inbound'] == False]

# Filter the DataFrame further to include only rows where the text meets the criteria
filtered_df = filtered_df[filtered_df['text'].apply(check_text)]

min_length_index = filtered_df['text'].str.len().idxmax()

# Get the row with maximum text length
min_length_row = filtered_df.loc[max_length_index]

# Extract text and its length
min_length_text = min_length_row['text']
min_length_text_length = len(min_length_text)

# Print the row with the maximum length of "text"
print("Row with maximum text length:")
print(min_length_row)
print()

# Print the text and its length
print("Text with maximum length:")
print("Text:", min_length_text)
print("Length:", min_length_text_length)


Row with maximum text length:
tweet_id                                           480988
author_id                                         AskeBay
inbound                                             False
created_at                 Fri Dec 01 13:26:40 +0000 2017
text                                              @229320
response_tweet_id                                     NaN
in_response_to_tweet_id                          480989.0
Name: 421999, dtype: object

Text with maximum length:
Text: @229320
Length: 7


#Tokenization

In [ ]:
df_tokenize = df_lower_case.copy();
df_tokenize['text_tokenized'] = df_lower_case['text_lower_case'].apply(lambda x:nltk.word_tokenize(x))
df_tokenize

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower_case,text_tokenized
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN,@applesupport causing the reply to be disregar...,"[@, applesupport, causing, the, reply, to, be,..."
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0,@105835 your business means a lot to us. pleas...,"[@, 105835, your, business, means, a, lot, to,..."
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN,@76328 i really hope you all change but i'm su...,"[@, 76328, i, really, hope, you, all, change, ..."
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0,@105836 livechat is online at the moment - htt...,"[@, 105836, livechat, is, online, at, the, mom..."
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0,@virgintrains see attached error message. i've...,"[@, virgintrains, see, attached, error, messag..."
...,...,...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0,@105860 i wish amazon had an option of where i...,"[@, 105860, i, wish, amazon, had, an, option, ..."
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN,they reschedule my shit for tomorrow https://t...,"[they, reschedule, my, shit, for, tomorrow, ht..."
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0,"@105861 hey sara, sorry to hear of the issues ...","[@, 105861, hey, sara, ,, sorry, to, hear, of,..."
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0,@tesco bit of both - finding the layout cumber...,"[@, tesco, bit, of, both, -, finding, the, lay..."


#Stemming and Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()


In [ ]:
df_lemmatization = df_tokenize.copy();
df_lemmatization['text_lemmatized'] = df_tokenize['text_tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df_lemmatization


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower_case,text_tokenized,text_lemmatized
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN,@applesupport causing the reply to be disregar...,"[@, applesupport, causing, the, reply, to, be,...","[@, applesupport, causing, the, reply, to, be,..."
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0,@105835 your business means a lot to us. pleas...,"[@, 105835, your, business, means, a, lot, to,...","[@, 105835, your, business, mean, a, lot, to, ..."
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN,@76328 i really hope you all change but i'm su...,"[@, 76328, i, really, hope, you, all, change, ...","[@, 76328, i, really, hope, you, all, change, ..."
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0,@105836 livechat is online at the moment - htt...,"[@, 105836, livechat, is, online, at, the, mom...","[@, 105836, livechat, is, online, at, the, mom..."
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0,@virgintrains see attached error message. i've...,"[@, virgintrains, see, attached, error, messag...","[@, virgintrains, see, attached, error, messag..."
...,...,...,...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0,@105860 i wish amazon had an option of where i...,"[@, 105860, i, wish, amazon, had, an, option, ...","[@, 105860, i, wish, amazon, had, an, option, ..."
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN,they reschedule my shit for tomorrow https://t...,"[they, reschedule, my, shit, for, tomorrow, ht...","[they, reschedule, my, shit, for, tomorrow, ht..."
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0,"@105861 hey sara, sorry to hear of the issues ...","[@, 105861, hey, sara, ,, sorry, to, hear, of,...","[@, 105861, hey, sara, ,, sorry, to, hear, of,..."
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0,@tesco bit of both - finding the layout cumber...,"[@, tesco, bit, of, both, -, finding, the, lay...","[@, tesco, bit, of, both, -, finding, the, lay..."


#Stemming

In [ ]:
stemmer = PorterStemmer()


In [ ]:
df_stemming = df_lemmatization.copy();
# Stem the tokenized text using NLTK
df_stemming['text_stemmed'] = df_lemmatization['text_lemmatized'].apply(lambda x: [stemmer.stem(word) for word in x])
df_stemming

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower_case,text_tokenized,text_lemmatized,text_stemmed
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN,@applesupport causing the reply to be disregar...,"[@, applesupport, causing, the, reply, to, be,...","[@, applesupport, causing, the, reply, to, be,...","[@, applesupport, caus, the, repli, to, be, di..."
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0,@105835 your business means a lot to us. pleas...,"[@, 105835, your, business, means, a, lot, to,...","[@, 105835, your, business, mean, a, lot, to, ...","[@, 105835, your, busi, mean, a, lot, to, u, ...."
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN,@76328 i really hope you all change but i'm su...,"[@, 76328, i, really, hope, you, all, change, ...","[@, 76328, i, really, hope, you, all, change, ...","[@, 76328, i, realli, hope, you, all, chang, b..."
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0,@105836 livechat is online at the moment - htt...,"[@, 105836, livechat, is, online, at, the, mom...","[@, 105836, livechat, is, online, at, the, mom...","[@, 105836, livechat, is, onlin, at, the, mome..."
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0,@virgintrains see attached error message. i've...,"[@, virgintrains, see, attached, error, messag...","[@, virgintrains, see, attached, error, messag...","[@, virgintrain, see, attach, error, messag, ...."
...,...,...,...,...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0,@105860 i wish amazon had an option of where i...,"[@, 105860, i, wish, amazon, had, an, option, ...","[@, 105860, i, wish, amazon, had, an, option, ...","[@, 105860, i, wish, amazon, had, an, option, ..."
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN,they reschedule my shit for tomorrow https://t...,"[they, reschedule, my, shit, for, tomorrow, ht...","[they, reschedule, my, shit, for, tomorrow, ht...","[they, reschedul, my, shit, for, tomorrow, htt..."
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0,"@105861 hey sara, sorry to hear of the issues ...","[@, 105861, hey, sara, ,, sorry, to, hear, of,...","[@, 105861, hey, sara, ,, sorry, to, hear, of,...","[@, 105861, hey, sara, ,, sorri, to, hear, of,..."
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0,@tesco bit of both - finding the layout cumber...,"[@, tesco, bit, of, both, -, finding, the, lay...","[@, tesco, bit, of, both, -, finding, the, lay...","[@, tesco, bit, of, both, -, find, the, layout..."


#Removal Of Stop Of Words

In [ ]:
stop_words = set(stopwords.words('english'))

# Remove stopwords from tokenized text using NLTK
df_stemming['text_without_stopwords'] = df_stemming['text_stemmed'].apply(lambda x: [word for word in x if word not in stop_words])

df_stemming

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower_case,text_tokenized,text_lemmatized,text_stemmed,text_without_stopwords
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN,@applesupport causing the reply to be disregar...,"[@, applesupport, causing, the, reply, to, be,...","[@, applesupport, causing, the, reply, to, be,...","[@, applesupport, caus, the, repli, to, be, di...","[@, applesupport, caus, repli, disregard, tap,..."
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0,@105835 your business means a lot to us. pleas...,"[@, 105835, your, business, means, a, lot, to,...","[@, 105835, your, business, mean, a, lot, to, ...","[@, 105835, your, busi, mean, a, lot, to, u, ....","[@, 105835, busi, mean, lot, u, ., pleas, dm, ..."
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN,@76328 i really hope you all change but i'm su...,"[@, 76328, i, really, hope, you, all, change, ...","[@, 76328, i, really, hope, you, all, change, ...","[@, 76328, i, realli, hope, you, all, chang, b...","[@, 76328, realli, hope, chang, 'm, sure, wo, ..."
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0,@105836 livechat is online at the moment - htt...,"[@, 105836, livechat, is, online, at, the, mom...","[@, 105836, livechat, is, online, at, the, mom...","[@, 105836, livechat, is, onlin, at, the, mome...","[@, 105836, livechat, onlin, moment, -, http, ..."
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0,@virgintrains see attached error message. i've...,"[@, virgintrains, see, attached, error, messag...","[@, virgintrains, see, attached, error, messag...","[@, virgintrain, see, attach, error, messag, ....","[@, virgintrain, see, attach, error, messag, ...."
...,...,...,...,...,...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0,@105860 i wish amazon had an option of where i...,"[@, 105860, i, wish, amazon, had, an, option, ...","[@, 105860, i, wish, amazon, had, an, option, ...","[@, 105860, i, wish, amazon, had, an, option, ...","[@, 105860, wish, amazon, option, get, ship, s..."
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN,they reschedule my shit for tomorrow https://t...,"[they, reschedule, my, shit, for, tomorrow, ht...","[they, reschedule, my, shit, for, tomorrow, ht...","[they, reschedul, my, shit, for, tomorrow, htt...","[reschedul, shit, tomorrow, http, :, //t.co/rs..."
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0,"@105861 hey sara, sorry to hear of the issues ...","[@, 105861, hey, sara, ,, sorry, to, hear, of,...","[@, 105861, hey, sara, ,, sorry, to, hear, of,...","[@, 105861, hey, sara, ,, sorri, to, hear, of,...","[@, 105861, hey, sara, ,, sorri, hear, issu, ,..."
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0,@tesco bit of both - finding the layout cumber...,"[@, tesco, bit, of, both, -, finding, the, lay...","[@, tesco, bit, of, both, -, finding, the, lay...","[@, tesco, bit, of, both, -, find, the, layout...","[@, tesco, bit, -, find, layout, cumbersom, re..."


#Punctuation

In [ ]:
punc_to_remove=string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
df_punc['text_without_punctuation'] = df_stemming['text_without_stopwords'].str.replace('[{}]'.format(re.escape(punc_to_remove)), '')
